In [9]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh

In [10]:
##First, let's create our web scraper
#The outline for the scraper can be found at:
#https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

#This scraper will take monthly data from Yahoo Finance over the last 5 years and convert it into a spreadsheet
#The only argument that will be necessary is the all upper-case ticker symbol (ie FB for facebook, LMT for lockheed Martin, etc.)
def monthly5yrscraper(x):
    url = 'https://finance.yahoo.com/quote/' + x + '/history?period1=1407733200&period2=1565499600&interval=1mo&filter=history&frequency=1mo'
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')

    col = []
    i=0


    for t in tr_elements[0]:
        i = i+1
        name=t.text_content()
        col.append((name, []))


    for j in range(1, len(tr_elements)):
        T = tr_elements[j]
    
        if len(T) != 7:
            continue
        k = 0
    
        for t in T.iterchildren():
            data=t.text_content()
        
            if k >0:
            
                try:
                    data=int(data)
                except:
                    pass
            col[k][1].append(data)
        
            k = k + 1
    Dict = {title:column for (title, column) in col}
    df = pd.DataFrame(Dict)
    
    return df

In [11]:
##Now that we've made our data scraper, we need to get a little more data.
#We'll look at the dow jones industrial companies
#To do this, we will need to scrape wikipedia for a list of all the stocks in the Dow Jones Industrial Average

url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//table[@id="constituents"]')[0]
symbol = tr_elements.xpath('//a[@class="external text"]/text()')
symbol

['www.djaverages.com',
 'MMM',
 'AXP',
 'AAPL',
 'BA',
 'CAT',
 'CVX',
 'CSCO',
 'KO',
 'DOW',
 'XOM',
 'GS',
 'HD',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'MCD',
 'MRK',
 'MSFT',
 'NKE',
 'PFE',
 'PG',
 'TRV',
 'UNH',
 'UTX',
 'VZ',
 'V',
 'WMT',
 'WBA',
 'DIS',
 'improve this section',
 'DIA',
 'ProShares',
 'DOG',
 'DDM',
 'DXD',
 'UDOW',
 'SDOW',
 'E-mini Dow ($5) Futures (YM)',
 'DJX',
 'Dow Record Book Adds Another First',
 '"Dow Jones Industrial Average Fact Sheet"',
 '"Dow Jones (DJIA) Historical Total Market Cap"',
 '"Problems with the Dow Jones Industrial Average-A Simple Unweighted Arithmetic Mean: An Issue Re-Revisited"',
 '10.21102/graf.2016.09.72.06',
 '"Why the Dow Doesn\'t Work - McLean Asset Management"',
 '"Ignore the Misleading Dow Jones Industrial Average"',
 '"Discover What Makes the Dow Jones Industrial Average Stupid"',
 '"Why the Dow Jones Industrial Average Is Useless"',
 '"Wayback Machine"',
 '"Dow Jones to change the composition of the Dow Jones Industrial Average

In [12]:
#It looks like our scraper brought back a few extra symbols from further down the page
#Fortunately, it looks like the list of stock tickers ends at 'improve this section' so we'll just take those
#Now we have our list with all of the symbols from the dow jones industrial average
djia = symbol[1:31]
djia

['MMM',
 'AXP',
 'AAPL',
 'BA',
 'CAT',
 'CVX',
 'CSCO',
 'KO',
 'DOW',
 'XOM',
 'GS',
 'HD',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'MCD',
 'MRK',
 'MSFT',
 'NKE',
 'PFE',
 'PG',
 'TRV',
 'UNH',
 'UTX',
 'VZ',
 'V',
 'WMT',
 'WBA',
 'DIS']

In [13]:
#Finally, let's create a loop that will create a different sheet for each stock in an excel workbook using our function 
#from the beginning
writer = pd.ExcelWriter('C:\\Users\\tilleymusprime\\Desktop\\stocks.xlsx', engine = 'xlsxwriter')
for i in djia:
    df=monthly5yrscraper(i)
    df.to_excel(writer, sheet_name=i)
writer.save()

In [14]:
##Another option is to combine every sheet into 1 sheet. This will be helpful if we want to do further analysis on all the
#stocks at once
df = pd.read_excel('C:\\Users\\tilleymusprime\\Desktop\\stocks.xlsx', sheet_name=0)
df.columns = ['Date',  'MMM Open', 'MMM High', 'MMM Low', 'MMM Close', 'MMM Adj Close', 'MMM Volume']
for i, j in zip(range(1, len(djia)), djia[1:]):
    sheet = pd.read_excel('C:\\Users\\tilleymusprime\\Desktop\\stocks.xlsx', sheet_name=i)
    sheet.columns = ['Date', j +' Open', j + ' High', j + 'Low', j + 'Close', j + 'Adj Close', j + 'Volume']
    df = pd.merge(df, sheet, how= 'outer', on='Date')
df.to_excel('C:\\Users\\tilleymusprime\\Desktop\\djia.xlsx')
df.head()

,Date,MMM Open,MMM High,MMM Low,MMM Close,MMM Adj Close,MMM Volume,AXP Open,AXP High,AXPLow,...,WBALow,WBAClose,WBAAdj Close,WBAVolume,DIS Open,DIS High,DISLow,DISClose,DISAdj Close,DISVolume
0,"Aug 01, 2019",174.79,177.67,158.04,162.10,162.10,"23,475,500",123.77,126.74,120.16,...,50.09,52.00,52.00,"40,242,800",143.34,144.53,132.26,135.75,135.75,"104,363,900"
1,"Jul 01, 2019",175.46,187.72,164.29,174.72,174.72,"62,578,400",125.30,129.34,123.62,...,53.66,54.49,54.49,"93,618,000",140.45,147.15,139.22,143.01,142.13,"162,287,400"
2,"Jun 01, 2019",159.75,174.56,159.32,173.34,173.34,"56,451,600",114.56,126.40,113.89,...,49.52,54.67,54.67,"122,039,500",132.02,143.51,131.49,139.64,138.78,"195,325,700"
3,"May 01, 2019",189.49,189.71,159.63,159.75,158.39,"94,604,800",117.47,120.88,114.56,...,49.31,49.34,48.93,"111,719,500",137.49,138.17,130.55,132.04,131.23,"208,497,300"
4,"Apr 01, 2019",209.89,219.75,186.55,189.51,187.90,"54,909,300",110.29,117.73,109.42,...,52.25,53.57,53.12,"201,758,600",111.59,142.37,111.38,136.97,136.13,"355,199,600"


In [15]:
df.to_excel('C:\\Users\\tilleymusprime\\Desktop\\djia.xlsx')